# Chapter 7 Random Field

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import *
import scipy

In [ ]:


# BCCB
def Circulant_Sample_2d(C_red, n1, n2):
    N = n1 * n2
    Lam = N * np.fft.ifft2(C_red)
    d = np.ravel(np.real(Lam))
    d_minus = np.maximum(- d, 0)
    if np.max(d_minus > 0):
        print(f'rho(D_minus)={np.max(d_minus)}')
    xi=np.random.randn(n1,n2) + 1j*np.random.randn(n1,n2)
    V=(Lam ** 0.5)*xi
    Z = np.fft.fft2(V) / sqrt(N)
    X=np.real(Z);    Y=np.imag(Z)
    return X, Y
